In [33]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [34]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Layout
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%matplotlib notebook
%use gromacs-5.1.4
%use pymol-1.8.4

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)


ChlB_AtNames = ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'OMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED']
ChlB_Q10 = np.array([-15.512,75.628,-57.844,-107.433,86.081,11.573,-99.670,7.454,7.749,77.389,3.351,-2.720,-5.754,4.859,0.650,-3.916,-73.990,85.402,-2.140,-5.618,108.571,14.838,1.978,24.357,-21.650,79.538,5.163,-5.803,-31.957,-13.413,-1.389,6.975,2.317,90.881,-88.170,-13.166,15.953,-98.277,-16.435,-28.234,-14.273,-0.035,7.743,-7.166,0.641,-4.526])
ChlB_Q10 = ChlB_Q10*1e-3*4.80326e-10

BChl_AtNames = ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'OBB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED', 'C1']
BChl_Q10 = np.array([-42.217,141.478,-14.988,-45.427,41.071,65.489,-134.776,19.419,2.723,17.043,11.656,-15.907,9.451,-5.063,5.421,-3.656,-28.216,87.663,-0.905,67.447,33.680,32.745,-25.091,37.208,19.037,16.855,80.984,-19.196,28.047,-116.550,7.787,0.863,-9.640,135.026,-111.514,-22.400,36.878,-200.720,-31.393,-35.896,-20.922,-2.346,-2.023,-8.885,7.086,-7.868,0.543])    
BChl_Q10 = BChl_Q10*1e-3*4.80326e-10

ChlA_AtNames = ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED', 'C1']
ChlA_Q10 = np.array([-21.674,106.779,-48.696,-98.725,72.726,31.683,-130.820,10.048,2.389,77.983,5.556,-1.050,0.735,-8.040,-1.330,7.091,-62.297,81.122,4.777,-9.204,106.271,16.963,10.681,34.947,-12.166,83.646,-7.420,-1.126,-44.008,-5.161,8.020,1.012,108.292,-110.812,-11.981,8.799,-125.044,-25.156,-19.250,-20.044,-11.238,5.979,-5.388,1.811,-5.256,-1.424])
ChlA_Q10 = ChlA_Q10*1e-3*4.80326e-10 # cgs base units for charge: cm^3/2 g^1/2 s^−1

Pheo_AtNames = ['CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED']
Pheo_Q10 = np.array([76.834,-38.800,-89.458,62.844,47.290,-86.246,7.560,-1.965,70.215,4.115,-2.753,1.651,-3.830,-1.585,2.947,-62.801,66.190,8.864,-9.269,90.283,15.078,9.556,28.399,-25.439,73.639,9.381,-20.445,-24.827,-3.833,6.785,-0.061,87.071,-87.246,-8.812,2.558,-94.919,-9.925,-16.035,-18.630,-8.844,4.351,-4.024,2.224,-5.226])
Pheo_Q10 = Pheo_Q10*1e-3*4.80326e-10

PorphAtNames=list()
PorphAtNames.append(ChlB_AtNames)
PorphAtNames.append(BChl_AtNames)
PorphAtNames.append(ChlA_AtNames)
PorphAtNames.append(Pheo_AtNames)

PorphQ10=list()
PorphQ10.append(ChlB_Q10)
PorphQ10.append(BChl_Q10)
PorphQ10.append(ChlA_Q10)
PorphQ10.append(Pheo_Q10)

PorphNames = ['Chl b', 'BChl a', 'Chl a', 'Pheo a']
PorphResNames = ['CLB', 'BCA', 'CLA', 'PHB']

#Chlorophyll Dipole Lengths = [Chlb, BChl, Chla, PheoA]
PorphDipLength = [3.60e-18,5.477e-18,4.2e-18,3.50e-18]


In [35]:
def classify_pigments():
    
    global xstruc
    
    Nporph = xstruc.nres
    err = False
    
    # Identify porph types
    for n in range(0,Nporph):
        resnum = xstruc.residxs[n]
        tstruc = struc[':'+str(resnum+1)]
        for p in range (0,len(PorphAtNames)):
            AtNames = PorphAtNames[p]
            Natoms = len(AtNames)
            match = True
            for at in range(0,Natoms):
                name = AtNames[at] 
                atstruc = tstruc['@'+name]
                if len(atstruc.coordinates)==0:
                    match = False
                    break
            if match==True:
                xstruc.typendcs[n] = p
                break
        if xstruc.typendcs[n]<0:
            print("Error! Chl " + str(resnum) + ' of chain ' + xstruc.chains[n] + ' was incomplete.')
            
    if np.min(xstruc.typendcs)<0:
        return False
    else:
        return True
        
# def fix_missing_atoms(mdstruc, resnumber):
#     for r in mdstruc.residues:
#         if(r.number==resnumber):
#             foundC = False
#             foundO = False
#             foundN = False
#             foundCA = False
#             for at in r:    
#                 if at.name=='C':
#                     foundC = True
#                 elif at.name=='O':
#                     foundO = True
#                 elif at.name=='N':
#                     foundN = True
#                 elif at.name=='CA':
#                     foundCA = True
                
#             if foundC * foundN * foundO * foundCA:
#                 print('Replacing with GLY')
#                 mdstruc.strip('(:'+str(r.idx+1)+')&!(@C,CA,N,O)')
#                 mdstruc.residues[r.idx].name = 'GLY'
#             else:
#                 print('Eliminating from the structure.')
#                 mdstruc.strip(':'+str(resnumber))
#             break



def mutate_residue(mdstruc, resnumber, mutresname):
    resndx = -1
    chain = ''
    for r in mdstruc.residues:
        if(r.number==resnumber):
            
            foundC = False
            foundO = False
            foundN = False
            foundCA = False
            for at in r:    
                if at.name=='C':
                    foundC = True
                elif at.name=='O':
                    foundO = True
                elif at.name=='N':
                    foundN = True
                elif at.name=='CA':
                    foundCA = True
                
            if foundC * foundN * foundO * foundCA:
                chain = r.chain
                resndx = r.idx
            else:
                print('Could not locate necessary backbone atoms: Eliminating from the structure.')
                print("")
                mdstruc.strip(':'+str(r.idx+1))
            break
            
    if resndx>=0:
        r = mdstruc.residues[resndx]
        print('Attempting to repair using PyMol mutagenesis wizard')
        
        # First replace with Gly: sometimes PyMol gets confused by incomplete side chains.
        mdstruc.strip('(:'+str(r.idx+1)+')&!(@C,CA,N,O)')
        mdstruc.residues[r.idx].name = 'GLY'
        mdstruc.write_pdb('gmx/tmp.pdb', altlocs='first')
        out = !{'pymol -qc misc/mutate.py -- gmx/tmp.pdb ' + chain + '/' + str(resnumber) + '/ ' + mutresname}

        # Check for errors
        error = False
        for line in out:
            if line.find('Error')>=0:
                error = True
                break
        if error:
            if(error):
                'PyMol Error:'
                for line in out:
                    print(line)
                print("Residue replaced with GLY.")
        else:
            print("Repair successful.")
            print("")
            mdstruc = pmd.load_file('gmx/tmp.pdb.mut')
        
    return mdstruc
    

def run_pdb2gmx(mdstruc, fname, lastPass):
    
    bondmax = 1.5*2
    
    success = True
    MissingAtomRegEx = re.compile('.*Residue(.*)named(.*)of a molecule in the input file was mapped to an entry in the topology database, but the atom(.*)used in that entry is not found in the input file')
    ResMisMatchRegEx = re.compile('.*Atom(.*)in residue[ ]+([a-zA-Z]+)[ ]+([0-9]+)[ ]+was not found in rtp entry(.*)with(.*)atoms while sorting atoms')
    ResNotFoundRegEx = re.compile('.*Residue(.*)not found in residue topology database')
    #out=!{"cd gmx; cp gmx/input.pdb.19 gmx/input.pdb; export GMX_MAXBACKUP=-1; gmx pdb2gmx -f input.pdb -o input.gro -ignh -water spce <<EOF\n8\nEOF"}
    #out=!{"cd gmx; rm input.gro; export GMX_MAXBACKUP=-1; gmx pdb2gmx -f "+fname+" -o input.gro -ignh -water spce <<EOF\n8\nEOF"}
    
    out=!{"cd gmx; rm input.gro; export GMX_MAXBACKUP=-1; gmx pdb2gmx -ff "+ ffselect.value + " -f " + fname + " -o input.gro -ignh -water " + waterselect.value}
    msg = ''
    
    madeTop = False
    MadeTopRegEx = re.compile('You have successfully generated a topology')
    for line in out:
        if re.match(MadeTopRegEx, line):
            madeTop = True
    
    WarnNonSeqChain = re.compile('.*Chain identifier \'(.*)\' is used in two non-sequential blocks.*')
    WarnOccup = re.compile('.*there were (.*) atoms with zero occupancy and (.*) atoms.*')
    #WarnLongBond = re.compile('.*Long bond ((.*)-(.*) = (.*) nm).*')
    WarnLongBond = re.compile('.*Long Bond.*\(([0-9]+)-([0-9]+).*\).*')
    WarnAtomMissing = re.compile('.*atom (.*) is missing in residue ([a-zA-Z]+)[ ]+([0-9]+) in the pdb file.*')

    
    for line in out:
        if(success and re.search('warning', line, flags=re.IGNORECASE)):
            if re.match(WarnAtomMissing, line):
                hit = re.match(WarnAtomMissing, line)
                resname = hit.group(2).strip()
                resnumber = int(hit.group(3).strip())
                print('Atoms were missing from residue '+resname+ ' ' + str(resnumber))
                #fix_missing_atoms(mdstruc, resnumber)
                mdstruc = mutate_residue(mdstruc, resnumber, resname)
                mdstruc.write_pdb('gmx/'+fname)
                success = False
            elif re.match(WarnLongBond, line):
                if madeTop:
                    hit = re.match(WarnLongBond, line)
                    at1tmp = int(hit.group(1))
                    at2tmp = int(hit.group(2))
                    at1 = min(at1tmp,at2tmp)
                    at2 = max(at1tmp,at2tmp)
                    
                    grostruc = pmd.load_file('gmx/input.gro')
                    x1 = grostruc.coordinates[at1-1]
                    x2 = grostruc.coordinates[at2-1]
                    dist = np.linalg.norm(x1 - x2)
                    
                    # If this is an amide bond, and the distance is unreasonably large, insert a 
                    # chain break between the residues.
                    if (dist>bondmax) and (grostruc[at1-1].name=='C') and (grostruc[at2-1].name=='N'):
                        res1 = grostruc[at1-1].residue.number
                        res2 = grostruc[at2-1].residue.number
                        print('Bond between atoms ' + str(at1) + ' and ' + str(at2) + ' was too long (' + str(dist) +').')
                        print('Identified atoms as belonging to residues ' + str(res1) + ' and ' + str(res2))
                        print('Inserting chain break between them.')
                        pritn("")
                        if res1<res2:
                            grostruc[at1-1].residue.ter = True
                        else:
                            grostruc[at2-1].residue.ter = True
                        grostruc.write_pdb('gmx/'+fname)
                        success = False
            elif re.match(WarnNonSeqChain, line):
                if lastPass:
                    print('Non-sequential chain warning')
            elif re.match(WarnOccup, line):
                if lastPass:
                    print('Occupancy warning')
            else:
                if lastPass:
                    print(line)
    
    if success:
        for line in out:
            
            if re.search('error', line, flags=re.IGNORECASE) and (len(msg)==0):
                success = False
                msg = line
                
            elif len(msg)>0:
                if len(line.strip())>0:
                    msg = msg + line + " "
                else:
                    if re.match(MissingAtomRegEx, msg):
                        hit = re.match(MissingAtomRegEx, msg)
                        resnumber = int(hit.group(1))
                        resname = hit.group(2).strip()
                        print('Atoms were missing from residue '+resname+ ' ' + str(resnumber))
                        #fix_missing_atoms(mdstruc, resnumber)
                        mdstruc = mutate_residue(mdstruc, resnumber, resname)
                        mdstruc.write_pdb('gmx/'+fname)

                    elif re.match(ResNotFoundRegEx, msg):
                        hit = re.match(ResNotFoundRegEx, msg)
                        resname = hit.group(1).strip().strip('\'')
                        print('Residue '+resname+ ' not available in the force field database.')
                        print('Eliminating from the structure.')
                        mdstruc.strip(':'+resname)
                        mdstruc.write_pdb('gmx/'+fname)

                    elif re.match(ResMisMatchRegEx, msg):
                        hit = re.match(ResMisMatchRegEx, msg)
                        resname = hit.group(2).strip()
                        resnum = int(hit.group(3).strip())
                        print('Residue ' +resname+ ' ' + str(resnum) + ' did not match the force field database entry.')
                        print('Eliminating from the structure.')
                        for r in mdstruc.residues:
                            if(r.number==resnum):
                                mdstruc.strip(":"+str(r.idx+1))
                        mdstruc.write_pdb('gmx/'+fname)
                    else:
                        if lastPass:
                            print(msg)

                    msg = ''
    return success
    
def ffgo_onclick(b):
    
    chainList = list()
    for r in struc.residues:
        if chainList.count(r.chain)==0:
            chainList.extend(r.chain)
            
    # First identify which chains should be displayed
    chainList = []
    # Loop through chain-selection check-boxes
    for cb in accordion.children[0].children[1:]:
        if cb.value==True:
            chainList.extend(cb.description)
    
    # Remove all pigments from input structure. They'll be checked separately and added back later. 
    pfstruc = struc[:]
    for ndx in np.sort(xstruc.residxs)[::-1]:
        pfstruc.strip(":"+str(ndx+1))
    
    failed = False
    for chain in chainList:
        fname = 'input_'+ chain + '.pdb'
        pfstruc.write_pdb('gmx/'+fname)
        mdstruc = pmd.load_file('gmx/'+fname)
        print("")
        print("********************************************")
        print('Preparing chain ' + str(chain) + ' for MD...')
        print("")
        mdstruc[chain,:,:].write_pdb('gmx/'+fname)
        
        maxTries = 100
        tries = 0
        tryAgain = True
        while tryAgain:
            tries += 1
            mdstruc = pmd.load_file('gmx/'+fname)
            mdstruc.write_pdb('gmx/' + fname + '.' + str(tries))
            if(run_pdb2gmx(mdstruc, fname, False)):
                tryAgain = False
                #run_pdb2gmx(mdstruc, fname, True)
                print('Chain ' + chain + ' successfully parameterized!')
                
            elif tries >= maxTries:
                tryAgain = False
                print('Failed to generate gmx input in ' + str(maxTries) + ' attempts')
                failed = True
        if failed:
            break
    if failed==False:
        fname = 'input.pdb'
        out=!{":>gmx/" + fname}
        for chain in chainList:
            out=!{'cat gmx/input_'+chain+'.pdb >> gmx/' + fname}
        
        out=!{'cat gmx/input.pdb'}
        mdstruc = pmd.load_file('gmx/'+fname)
        if(run_pdb2gmx(mdstruc, fname, True)):
            print('Successfully parameterized all chains!')
        

class ExcStruc:
    def __init__(self):
        self.nres = 0
        self.residxs = []
        self.resnums = []
        self.resclasses = []
        self.typendcs = []
        self.chains = []
        self.ham = []
        self.dips = []
        self.cents = []
        self.couplings = []
        self.struc = pmd.structure.Structure()
        self.nsd_pos = []
            
    def reset(self):
        self.nres = 0
        self.residxs = []
        self.resnums = []
        self.resclasses = []
        self.typendcs = []
        self.chains = []
        self.ham = []
        self.dips = []
        self.cents = []
        self.struc = pmd.structure.Structure()
        
    def build_structure(self):
        coords = []
        for n in range(0, self.nres):
            cent = xstruc.cents[n,:]
            dip = xstruc.dips[n,:]
            vStart = cent - 1.25*dip
            vStop = cent + 1.25*dip
            
            Nats = 10
            for a in range(0, Nats+1):
                xyz = (float(a)/float(Nats))*vStart + (1 - float(a)/float(Nats))*vStop
                self.struc.add_atom(pmd.topologyobjects.Atom(name='N'), 'Dip', self.residxs[n], chain=self.chains[n])
                coords.append(xyz)
        if len(coords)>0:
            self.struc.coordinates = np.array(coords)
            

class NGLRepList:
    def __init__(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def reset(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def append(self, nrep):
        self.names.append(nrep.name)
        ptext = {"type": nrep.type, "params": {"color": nrep.color, "sele": nrep.selection, "opacity": str(nrep.opacity)}}
        self.params.append(ptext)
        self.reps.append(nrep)
        
class NGLRep:
    def __init__(self, name, rtype, sel, col, opac):
        self.name = name
        self.type = rtype
        self.selection = sel
        self.color = col
        self.opacity = opac

def check_atoms(RefChlA_AtNames, QuerChlA_AtNames):
    ismatch = True
    for atnm in RefChlA_AtNames:
        if(QuerChlA_AtNames.count(atnm)!=1):
            ismatch = False
    return ismatch

def find_porph(pmdstruc, xtruc):
    PORatList = list(['NA', 'NB', 'NC', 'ND'])
    for n in range(0, len(pmdstruc.residues)):
        r = pmdstruc.residues[n]
        nmlist = list()
        for at in r:
            nmlist.append(at.name)
        if(check_atoms(PORatList, nmlist)):
            if(nmlist.count('MG')==1):
                xstruc.residxs.append(n)
                xstruc.resnums.append(r.number)
                xstruc.nres += 1 
                xstruc.resclasses.append('CHL')
            else:
                xstruc.residxs.append(n)
                xstruc.resnums.append(r.number)
                xstruc.nres += 1 
                xstruc.resclasses.append('PHO')
            xstruc.chains.append(r.chain)
    xstruc.typendcs = np.zeros((xstruc.nres,), dtype='int')
    return


def find_dipoles(struc,xtruc):
    ResNums = xtruc.residxs
    DipMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        for at in r:
            if at.name=='NB':
                NB = struc.coordinates[at.idx]
            if at.name=='ND':
                ND = struc.coordinates[at.idx]
        DipMat[n,:] = NB - ND
    xtruc.dips = DipMat

def find_centers(struc, xtruc):
    ResNums = xtruc.residxs
    CentMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        cent = 0.0
        for at in r:
            if at.name=='NB':
                cent += 0.5*struc.coordinates[at.idx]
            if at.name=='ND':
                cent += 0.5*struc.coordinates[at.idx]
        CentMat[n,:] = cent
    xtruc.cents = CentMat

def sync_widgets_to_rep(rep):
    seldrop.value = rep.name
    styledrop.value = rep.type
    colordrop.value = rep.color
    opacslide.value = int(rep.opacity*100)
    
def sync_rep_to_widgets():
    global repList
    num = repList.names.index(seldrop.value)
    rep = repList.reps[num]
    rep.type = styledrop.value
    rep.name = seldrop.value
    rep.color = colordrop.value
    rep.opacity = opacslide.value*0.01
    ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
    repList.params[num] = ptext
    pdbview.set_representations(repList.params)
    
def std_rep(nglview, struc, xstruc):
    global repList
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    chltxt = ''
    photxt = ''
    for n in range(0, len(xstruc.residxs)):
        if xstruc.resclasses[n]=='CHL':
            if len(chltxt)>0:
                chltxt += ' or '
            chltxt += str(xstruc.residxs[n]+1)
        if xstruc.resclasses[n]=='PHO':
            if len(photxt)>0:
                photxt += ' or '
            photxt += str(xstruc.residxs[n]+1)
            
    chainList = list()
    chainString = ''
    for r in struc.residues:
        if chainList.count(r.chain)==0:
            chainList.extend(r.chain)
            if len(chainString)>0:
                chainString += " OR "
            chainString += ":" + r.chain
    
    repList.append(NGLRep("Protein", "cartoon", "(protein)" + "AND (" + chainString + ")", "grey", 0.2))
    if len(chltxt)>0:
        repList.append(NGLRep("Chlorophyll", "licorice", "(" + chltxt + ")" + "AND (" + chainString + ")", "green", 1.0))
    if len(photxt)>0:
        repList.append(NGLRep("Pheophytin", "licorice", "(" + photxt + ")" + "AND (" + chainString + ")", "blue", 1.0))
    
    pdbview.set_representations(repList.params)
    
    seldrop.options=repList.names
    seldrop.disabled=False
    
    styledrop.disabled=False
    
    colordrop.disabled=False
    opacslide.disabled=False
    sync_widgets_to_rep(repList.reps[0])
    
    if xstruc.nres>0:
        pdbview.add_trajectory(xstruc.struc)
        pdbview.set_representations([{"type": "licorice", "params": {"color": "red", "sele": "("+chainString+")", "opacity": "1", "radius": "0.35"}}], component=1)
    
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

pdbidtxt = widgets.Label(value='Enter PDB ID:', width='4cm')

pdbgo = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    layout = widgets.Layout(width='2cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)

#pdbup = hublib.ui.FileUpload("", "Upload a PDB file for display and processing", basic=True, width='2cm', maxsize='500M')
pdbup = widgets.FileUpload(
    accept='.pdb',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

pdbuptxt = widgets.Label(value='Or upload a PDB file:', width='4cm')

Output = widgets.Output()


tresp_go = widgets.Button(
    description='TrESP',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to calculate inter-site couplings using the Transition Electmrostatic Potential (TrESP) method',
    layout = widgets.Layout(width='2cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)


def tresp_onclick(b):
    
    global xstruc

    Nporph = xstruc.nres
    AtomPosList=list()
    AtomQList=list()
    CoupMat = np.zeros((Nporph,Nporph))
    h = 6.62607015e-34 #J*s
    c = 2.998e10 #cm/s
    
    if classify_pigments()==False:
        xstruc.couplings = []
        return False
    else:
        ResTypeNDX = xstruc.typendcs.copy()
        for n in range(0,Nporph):
            resnum = xstruc.residxs[n]
            tstruc = struc[':'+str(resnum+1)]
            p = ResTypeNDX[n]
            print('Pigment ' + str(resnum) + ' identified as ' + PorphNames[p])
            AtNames = PorphAtNames[p]
            Q10 = PorphQ10[p]
            Natoms = len(AtNames)
            AtomPos = np.zeros((Natoms,3))
            AtomQ = np.zeros((Natoms,))
            dip = np.zeros((3,)) 
            for at in range(0,Natoms):
                name = AtNames[at]
                atstruc = tstruc['@'+name]
                AtomPos[at, :] = (atstruc.coordinates[0])*(1e-8) # cm
                AtomQ[at] = Q10[at] # cm^3/2 g^1/2 s^−1
                dip += AtomPos[at,:]*AtomQ[at]
                
            AtomPosList.append(AtomPos)
            AtomQList.append(AtomQ)
                    
            if np.linalg.norm(dip)>0:
                AtomQList[n] *= PorphDipLength[p]/np.linalg.norm(dip)
    
        for m in range (0,Nporph):
            for n in range(0,m):
                for atm in range (0, len(AtomQList[m])):
                    for atn in range(0, len(AtomQList[n])):
                        Rmn = AtomPosList[m][atm,:] - AtomPosList[n][atn,:] 
                        rmn = np.linalg.norm(Rmn) 
                        CoupMat[m,n]+= (((AtomQList[m][atm]*AtomQList[n][atn])/(rmn))*(1e-7))/(h*c)

        CoupMat = CoupMat + np.transpose(CoupMat)
        xstruc.couplings = CoupMat
        specgo.disabled = False
        return True

tresp_go.on_click(tresp_onclick)



def pdbwrite_onclick(b):
    fname = "pdb/" + accordion.children[1].children[0].children[0].value
    # First identify which chains should be displayed
    # Loop through chain-selection check-boxes
    strucList = []
    for cb in accordion.children[0].children[1:]:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
        selstruc.write_pdb(fname)
        


def clear_stage(view, xstruc):

    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    view._update_component_auto_completion()


# def pdbup_onclick(widg, fname):
#     load_structure(fname[-1])
#     widg.reset()

def pdbup_on_value_change(change):
    for item in pdbup.value:
        fname = item
    
    with open("pdb/"+fname, "wb") as fp:
        fp.write(pdbup.value[fname]["content"])
    fp.close()
    load_structure("pdb/"+fname)
    
        
def pdbgo_onclick(b):
    fname = 'pdb/' + pdbid.value + '.pdb'
    load_structure(fname)
    
def load_structure(fname):
    global struc
    global estruc
    
    specgo.disabled = True
    
    if os.path.isfile(fname):
        fid = fname.split('/')[-1]
        fid = fid.split('.')[0]
        pdbid.value=fid
        
    else:
        url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
        r = requests.get(url, allow_redirects=True)
        if(r.status_code==200):
            fname = 'pdb/'+pdbid.value+'.pdb'
            wfd = open(fname, 'wb')
            wfd.write(r.content)
            wfd.close()
            pdboutput.value = ''
            
        else:
            print('Invalid PDB code. Please try again.')
            
    
        
    if os.path.isfile(fname):
        #clear_stage(pdbview, xstruc)
        while len(pdbview._ngl_component_ids)>0:
            pdbview.remove_component(pdbview._ngl_component_ids[0])
        
        accordion.children = []
        repList.reset()
        xstruc.reset()
        
        struc = pmd.load_file(fname)
        find_porph(struc, xstruc)
        find_dipoles(struc, xstruc)
        find_centers(struc, xstruc)
        xstruc.build_structure()
        std_rep(pdbview, struc, xstruc)
        chainList = list()
        chainString = ''
        for r in struc.residues:
            if chainList.count(r.chain)==0:
                chainList.extend(r.chain)
                chainString += r.chain
                
        chwidgList = [widgets.HBox([widgets.Button(description='All',tooltip='Click to select all chains', layout = widgets.Layout(width='1.5cm')),
                    widgets.Button(description='None',tooltip='Click to deselect all chains',layout = widgets.Layout(width='1.5cm'),)])]
        
        chwidgList[0].children[0].on_click(select_all_chains)
        chwidgList[0].children[1].on_click(deselect_all_chains)
        
        for chain in chainList:
            chwidgList.append(widgets.Checkbox(description=chain, value=True,indent=False))
            chwidgList[-1].observe(chain_box_on_change)
        accordion.children = [widgets.VBox(chwidgList)]
        
        deffnm = pdbid.value + '_' + chainString + '.pdb'
        pdbwrwidglist = [widgets.HBox([widgets.Text(value=deffnm, description='File Name:', disabled=False, layout=widgets.Layout(width='5cm')),widgets.Button(description='Write',tooltip='Click to write PDB file',layout=widgets.Layout(width='1.5cm'),)])]
        pdbwrwidglist[0].children[1].on_click(pdbwrite_onclick)
        
        accordion.children = tuple(list(accordion.children) + [widgets.VBox(pdbwrwidglist)])
        accordion.children = tuple(list(accordion.children) + [widgets.VBox([tresp_go])])
        set_md_state(False)
        
    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

def select_all_chains(b):
    for cb in accordion.children[0].children[1:]:
        cb.value = True
        
def deselect_all_chains(b):
    for cb in accordion.children[0].children[1:]:
        cb.value = False
        
        
def chain_box_on_change(b):
    global repList
    if b['type'] == 'change' and b['name'] == 'value':
        
        # First identify which chains should be displayed
        chainString = ''
        chainList = []
        chainChars = ''
        # Loop through chain-selection check-boxes
        for cb in accordion.children[0].children[1:]:
            if cb.value==True:
                chainList.extend(cb.description)
                if len(chainString)>0:
                    chainString += " OR "
                chainString += ":" + cb.description
                chainChars += cb.description
                
        for num in range(0, len(repList.reps)):
            rep = repList.reps[num]
            selList = rep.selection.split('AND')
            selText = ''
            for item in selList:
                if item.find(":")==-1:
                    if len(selText)>0:
                        selText += " AND "
                    selText += item.strip()
            
            if len(chainString)>0:
                selText += " AND (" + chainString + ")"
            else:
                # We assume no structure has chain ID XXXX...
                selText += " AND (:XXXX)"
                
            rep.selection = selText
            ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
            repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
        if len(chainString)>0:
            pdbview.set_representations([{"type": "licorice", "params": {"color": "red", "sele": "("+chainString+")", "opacity": "1", "radius": "0.35"}}], component=1)
            accordion.children[1].children[0].children[0].value = pdbid.value + "_" + chainChars + '.pdb'
            accordion.children[1].children[0].children[1].disabled = False
            accordion.children[2].children[0].disabled = False
            set_md_state(False)
        else:
            pdbview.set_representations([{"type": "licorice", "params": {"color": "red", "sele": "(:XXXX)", "opacity": "1", "radius": "0.35"}}], component=1)
            accordion.children[1].children[0].children[0].value = ''
            accordion.children[1].children[0].children[1].disabled = True
            accordion.children[2].children[0].disabled = True
            set_md_state(True)
            
def seldrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        rep = repList.reps[repList.names.index(seldrop.value)]
        sync_widgets_to_rep(rep)
        
def styledrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.type = styledrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
def colordrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.color = colordrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
def opacslide_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.opacity = opacslide.value*0.01
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)

repList = NGLRepList()
seldrop = widgets.Dropdown(
    options=repList.names,
    #value='Protein',
    description='Selection:',
    disabled=True,
)

styledrop = widgets.Dropdown(
    options=['cartoon', 'licorice', 'spacefill'],
    description='Style:',
    disabled=True,
)

colordrop = widgets.Dropdown(
    options=['chain', 'red', 'green', 'blue', 'grey'],
    description='Color:',
    disabled=True,
)

opacslide = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Opacity:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

seldrop.observe(seldrop_on_change)
styledrop.observe(styledrop_on_change)
colordrop.observe(colordrop_on_change)
opacslide.observe(opacslide_on_change)


accordion = widgets.Accordion(children=[])
accordion.set_title(0, 'Chain Selection')
accordion.set_title(1, 'Write PDB')
accordion.set_title(2, 'Calculate Couplings')
accordion.set_title(3, 'Prepare for MD')

#pdbuptext = widgets.Label(value="Or upload a file:", layout=widgets.Layout(width='3.5cm', display="flex", justify_content='flex-end'))

pdbgo.on_click(pdbgo_onclick)

pdbup.observe(pdbup_on_value_change, 'value')
pdbid.on_submit(pdbgo_onclick)
pdbview = nv.NGLWidget()
pdbview._set_size('400px', '400px')


pdbbox = widgets.VBox([widgets.HBox([pdbidtxt, pdbid, pdbgo]), widgets.HBox([pdbuptxt, pdbup])])
dispbox = widgets.VBox([pdbbox, seldrop, styledrop, colordrop, opacslide, accordion])

strucbox = widgets.HBox([pdbview, dispbox])

specview = widgets.Output()

abscb = widgets.Checkbox(
    value=True,
    description='Absorption',
    disabled=False
)

emicb = widgets.Checkbox(
    value=False,
    description='Fluorescence',
    disabled=False
)

cdcb = widgets.Checkbox(
    value=False,
    description='CD',
    disabled=False
)

hbcb = widgets.Checkbox(
    value=False,
    description='Hole Burning',
    disabled=False
)

def update_spec_lims():
    
    lines = [abs_line, emi_line, cd_line, hb_line]
    myinf = 1e+100
    mny = myinf
    mxy = -myinf
    for line in lines:
        if line.get_visible()==True:
            mny = min(mny, np.min(line.get_ydata()))
            mxy = max(mxy, np.max(line.get_ydata()))
    if (mny<myinf) and (mxy>-myinf) and (mny!=mxy):
        specax.set_ylim(mny, mxy)
        
    mnx = len(vaxis) - 1
    mxx = 0
    cutoff = 1e-4
    for line in lines:
        if line.get_visible()==True:
            ndcs = np.where(np.abs(line.get_ydata())>cutoff)[0]
            if len(ndcs>0):
                mnx = min(mnx, np.min(ndcs))
                mxx = max(mxx, np.max(ndcs))
    if mxx>mnx:
        specax.set_xlim(vaxis[mnx], vaxis[mxx])
        update_ax2()
    

def update_spec_visibility(b):
    if abscb.value==True:
        abs_line.set_visible(True)
    else:
        abs_line.set_visible(False)
        
    if emicb.value==True:
        emi_line.set_visible(True)
    else:
        emi_line.set_visible(False)
        
    if cdcb.value==True:
        cd_line.set_visible(True)
    else:
        cd_line.set_visible(False)
        
    if hbcb.value==True:
        hb_line.set_visible(True)
    else:
        hb_line.set_visible(False)
        
    update_spec_lims()
        
abscb.observe(update_spec_visibility)
emicb.observe(update_spec_visibility)
cdcb.observe(update_spec_visibility)
hbcb.observe(update_spec_visibility)

def specgo_onclick(b):
    Nporph = xstruc.nres
    
    if (np.shape(xstruc.couplings)[0]!=Nporph) or (np.shape(xstruc.couplings)[1]!=Nporph):
        print('Please calculate coupling constants before calculate spectra.')
        return 0
    else:
        abs_spec = np.zeros(np.shape(vaxis))
        Ham = xstruc.couplings + np.eye(Nporph)*14925.37
        eVals,eVecs = np.linalg.eigh(Ham)
        eMu = np.matmul(np.transpose(eVecs), xstruc.dips)

        for n in range(0,Nporph):
            ndx = np.where(np.abs(vaxis-eVals[n])==np.min(np.abs(vaxis-eVals[n])))[0][0]
            abs_spec[ndx] += np.linalg.norm(eMu[n])**2/Nporph #mu is a 3 element vector

        abs_spec = gaussian_filter(abs_spec, sigma=20)
        abs_line.set_ydata(abs_spec)

        update_spec_lims()
    
specgo = widgets.Button(description='Calculate', disabled=True)
specgo.on_click(specgo_onclick)
spec_checks = widgets.VBox([specgo, abscb, emicb, cdcb, hbcb])
specbox = widgets.HBox([specview, spec_checks])

mdview = nv.NGLWidget()
mdview._set_size('400px', '400px')
ffgo = widgets.Button(
        description='Prepare MD',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to prepare structure for molecular dynamics (MD)',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
ffgo.on_click(ffgo_onclick)

fflabel = widgets.Label(value='Force Field')
ffselect = widgets.RadioButtons(
    options=['charmm27', 'oplsaa'],
    value='charmm27',
    #description='Force field for MD',
    disabled=True
)

waterlabel = widgets.Label(value='Water Model')
waterselect = widgets.RadioButtons(
    options=['spce', 'tip3p', 'spc'],
    value='spce',
    disabled=True
)

def set_md_state(choice):
    ffgo.disabled = choice
    ffselect.disabled = choice
    waterselect.disabled = choice

ffbox = widgets.VBox([fflabel, ffselect, waterlabel, waterselect])

mdacc = widgets.Accordion(children=[])
mdacc.set_title(0, 'Force Field')
mdacc.set_title(1, 'Equilibration')
mdacc.set_title(2, 'Production')
mdacc.children = [ffbox]

mdbox = widgets.HBox([mdview, widgets.VBox([mdacc, ffgo])])

maintab = widgets.Tab([strucbox, mdbox, specbox])
maintab.set_title(0, 'Structure')
maintab.set_title(1, 'Molecular Dynamics')
maintab.set_title(2, 'Spectrum')

display(pdboutput)
display(maintab)


def format_coord(x, y):
    return 'x=%1.1f, y=%1.2f'%(x, y)

def lamba2vbar(lamb):
    vbar = (1.0e+7)/lamb
    
def vbar2lambda(vbar):
    return (1.0e+7)/vbar

def update_ax2():
    specax2.set_xlim(specax.get_xlim())
    xticks2 = (1.0e7)/(specax.get_xticks())
    
    dlambda = -np.mean(np.diff(xticks2))
    ndec = max(-int(np.ceil(np.log10(dlambda))), 0)
    fmt = "{:." + str(ndec) + "f}"
    xticklabels2 = []
    for tick in xticks2:
        xticklabels2.append(fmt.format(tick))
    
    specax2.set_xticks((1.0e7)/xticks2)
    specax2.set_xticklabels(xticklabels2)
    
vaxis = np.arange(14000, 18000)
#vaxis = np.arange(14000, 14000.1, 0.1)
with specview:
    %matplotlib notebook
    fig = plt.figure("Calculated Spectra", figsize=(4,4))
    abs_line, = plt.plot(vaxis, 0*vaxis)
    emi_line, = plt.plot(vaxis, 0*vaxis)
    cd_line, = plt.plot(vaxis, 0*vaxis)
    hb_line, = plt.plot(vaxis, 0*vaxis)
    update_spec_visibility(0)
    specax = plt.gca()
    specax.set_xlabel('$\\omega/2\\pi$c (cm$^{-1}$)')
    specax.set_ylabel('Signal (arbitrary units)')
    specax.set_yticklabels([])
    specax.format_coord = format_coord
    specax2 = specax.twiny()
    specax2.xaxis.tick_top()
    specax2.set_xlabel('$\\lambda$ (nm)')
    update_ax2()
    
    # ax2 = ax.secondary_xaxis('top', functions=(vbar2lambda, lambda2vbar))
    # secax.set_xlabel('$\\lambda$ (nm))')
    plt.tight_layout()
    
xstruc = ExcStruc()
struc = pmd.structure.Structure()

HTML(value='', placeholder='')

In [37]:
def nsd_onclick(b):
    
    # Make xstruc available to change in this function
    global xstruc

    # How many pigments are there?
    Nporph = xstruc.nres
    
    # Check that we know what all pigment are
    if classify_pigments()==False:
        xstruc.couplings = []
        return False
    
    else:
        # ResTypeNDX stores an integer for each pigment to identify its type
        #  0 = Chl b
        #  1 = BChl a
        #  2 = Chl a
        #  3 = Pheo A
        ResTypeNDX = xstruc.typendcs.copy()
        
        # Loop through pigments
        for n in range(0,Nporph):
            
            # Residue number for pigment in parmed structure
            resnum = xstruc.residxs[n]
            
            # Parmed structure object that contains ONLY this pigment
            tstruc = struc[':'+str(resnum+1)]
            
            # p is the index for this pigment type in PorphNames, PorphAtNames, etc.
            p = ResTypeNDX[n]
            print('Pigment ' + str(resnum) + ' identified as ' + PorphNames[p])
            
            # Stores atom names for pigment
            AtNames = PorphAtNames[p]
            
            # Stores number of atoms in pigment
            Natoms = len(AtNames)
            
            # This will store the coordinates of the pigment's atoms
            AtomPos = np.zeros((Natoms,3))
            
            names = ['NA', 'NB', 'NC', 'ND', 'CHB', 'CHC', 'CHD', 'CHA', 'C1A', 'C4A', 'C1B', 'C4B', 'C1C', 'C4C', 'C1D', 'C4D', 'C2A', 'C3A', 'C2B', 'C3B', 'C2C', 'C3C', 'C2D', 'C3D']
            
            # Populate AtomPos array
            for atm in names:
                for at in range(0,Natoms):
                    name = AtNames[at]
                    if name == atm:
                        atstruc = tstruc['@'+name]
                        AtomPos[at, :] = (atstruc.coordinates[0])*(1e-8) # cm
                        nsdcoords = name+" "+str(atstruc.coordinates[0])
                        xstruc.nsd_pos.append(nsdcoords)
                        #print (name,AtomPos[at, :])
            print (xstruc.nsd_pos)
                
        return True

In [38]:
nsd_onclick(1)

Pigment 699 identified as Chl a
['NA [13.865 42.232 -4.392]', 'NB [12.865 39.899 -2.658]', 'NC [10.749 39.271 -4.668]', 'ND [11.78  41.365 -6.365]', 'CHB [14.717 41.458 -2.197]', 'CHC [11.2   38.077 -2.525]', 'CHD [ 9.972 39.812 -6.993]', 'CHA [13.502 43.209 -6.64 ]', 'C1A [14.207 43.121 -5.415]', 'C4A [14.758 42.277 -3.351]', 'C1B [13.89  40.371 -1.888]', 'C4B [12.306 38.809 -2.03 ]', 'C1C [10.555 38.242 -3.772]', 'C4C [10.009 39.013 -5.814]', 'C1D [10.874 40.869 -7.284]', 'C4D [12.447 42.358 -6.995]', 'C2A [15.361 43.984 -4.98 ]', 'C3A [15.743 43.41  -3.595]', 'C2B [13.972 39.572 -0.679]', 'C3B [12.99  38.615 -0.738]', 'C2C [ 9.481 37.413 -4.267]', 'C3C [ 9.137 37.883 -5.519]', 'C2D [10.909 41.67  -8.503]', 'C3D [11.907 42.582 -8.306]']
Pigment 700 identified as Chl a
['NA [13.865 42.232 -4.392]', 'NB [12.865 39.899 -2.658]', 'NC [10.749 39.271 -4.668]', 'ND [11.78  41.365 -6.365]', 'CHB [14.717 41.458 -2.197]', 'CHC [11.2   38.077 -2.525]', 'CHD [ 9.972 39.812 -6.993]', 'CHA [13.502

True